In [1]:
kmodes <- read.csv(file='data/results/meteorites_clustered.csv')
kmodes <- as.data.frame(kmodes)

In [45]:
kmodes_summary <- by(kmodes, kmodes$Cluster, summary)

In [46]:
kmodes_summary

kmodes$Cluster: 0
    Cluster       mass         fell_found        year         latitude     
 Min.   :0   Min.   :      0   Fell : 297   Min.   :1491   Min.   :-44.82  
 1st Qu.:0   1st Qu.:     67   Found:4718   1st Qu.:1995   1st Qu.: 18.91  
 Median :0   Median :    238                Median :2001   Median : 21.04  
 Mean   :0   Mean   :   5562                Mean   :1992   Mean   : 19.67  
 3rd Qu.:0   3rd Qu.:    999                3rd Qu.:2004   3rd Qu.: 28.46  
 Max.   :0   Max.   :4000000                Max.   :2013   Max.   : 66.00  
                                                                           
   longitude             group           chondrite            type     
 Min.   :-157.7833   H      :3737   Achondrite: 321   Iron      :   0  
 1st Qu.:   0.1451   LL     : 374   Chondrite :4694   Stony     :4969  
 Median :  16.4877   L      : 318                     Stony-Iron:  46  
 Mean   :  14.2021   Eucrite:  74                                      
 3rd Qu.:  54.

In [39]:
## SETUP
library(shiny)
library(leaflet)
library(ggplot2)
library(dplyr)
library(tidyr)
library(reshape2)
library(dygraphs)
library(TTR)
library(rvest)
library(xts)
options(scipen=999)

#Data Import
#All Meteorites
meteorites <- read.csv(file='data/results/meteorites_lc.csv')
meteorites <- meteorites[, c(2:14)]
colnames(meteorites) <- c('Name',
                          'Fell_or_Found',
                          'Year',
                          'Mass',
                          'longitude',
                          'latitude',
                          'Land_Cover',
                          'Chondrite',
                          'Type',
                          'Level_3', 
                          'Level_4', 
                          'Group',
                          'recclass')

#Prob Density Function Land Cover
pdf <- read.csv(file = 'data/results/pdf.csv')
pdfmelt<-melt(pdf,id="x_axis")

LC_names  <- list("Oceans/Water",
                  "Cultivated Land",
                  "Pasture/Land for Grazing",
                  "Tundra",
                  "Wooded Tundra",
                  "Boreal Forest",
                  "Cool Conifer Forest",
                  "Temperate Mixed Forest",
                  "Temperate Decidious Forest",
                  "Grassland/Steppe",
                  "Hot Desert",
                  "Savanna",
                  "Tropical Woodland",
                  "Tropical Forest")

LC_values  <- list("water_pdf",
                   "cultivated_pdf",
                   "pasture_pdf",
                   "tundra_pdf",
                   "wooded_tundra_pdf",
                   "boreal_forest_pdf",
                   "conifer_forest_pdf",
                   "temperate_mixed_forest_pdf",
                   "temperate_decidous_forest_pdf",
                   "grasssland_pdf",
                   "desert_pdf",
                   "savanna_pdf",
                   "tropical_wooded_pdf",
                   "tropical_forest_pdf")

LC_names2  <- list("Boreal Forest",
                   "Cool Conifer Forest",
                   "Cultivated Land",
                   "Grassland/Steppe",
                   "Hot Desert",
                   "Oceans/Water",
                   "Pasture/Land used for Grazing",
                   "Savanna",
                   "Scrubland",
                   "Temperate Decidious Forest",
                   "Temperate Mixed Forest",
                   "Tropical Forest",
                   "Tropical Woodland",
                   "Tundra", 
                   "Warm Mixed Forest",
                   "Wooded Tundra")

#Timeline
falls <- subset(meteorites, Fell_or_Found == 'Fell')
worldpop <- read.csv("data/worldpop/worldpop.csv")
worldpop <- worldpop %>% rename(pop = World.Population.over.12000.years..various.sources..2019..,
                                year = Year)
avg_worldpop <- worldpop %>% group_by(year) %>% summarize(avg=mean(pop))
avg_worldpop <- as.data.frame(avg_worldpop)
avg_worldpop <- avg_worldpop[avg_worldpop$year >= 800 & avg_worldpop$year <= 2013,]
fall_count <- count(falls, vars = Year)
fall_count$decade <- floor(fall_count$vars/10)*10
falls_decade <- aggregate(fall_count$n, by=list(Category=fall_count$decade), FUN=sum)
for (i in 1:nrow(falls_decade)){
    x <- falls_decade$Category[i]
    w <- avg_worldpop$year
    closestVal = w[which.min(abs(w-x))]
    pop <- avg_worldpop[avg_worldpop$year == closestVal,]$avg
    falls_decade$pop[i] <- pop}
falls_decade <- falls_decade %>% rename(
    decade = Category,
    meteorite_falls = x,
    population = pop)
falls_decade$falls_per_1M <- falls_decade$meteorite_falls/(falls_decade$population/1000000)
falls_decade <- falls_decade[-c(44), ] 

#Falls and Finds Gridsquares
gsfellfound <- read.csv("data/results/gridsquarefellfound.csv")

#Correlation
meteorites_corr <- read.csv(file='data/results/all.csv')
meteorites_corr <-subset(meteorites_corr, select=-c(1,2,3,5,9,10,11,12,13,15))
corrlist <- list( "Mass"= "mass",
                  "Year"= "year",
                  "Latitude"= "latitude",
                  "Longitude"= "longitude",
                  "Land Cover Type" = "lc_sample")

#KModes
kmodes <- read.csv(file='data/results/meteorites_clustered.csv')
kmodes <- subset(kmodes, select=-c(1))
kmodes <- as.data.frame(kmodes)

#Bonus
meteorites_full <- read.csv(file='data/results/full_dataset.csv')
meteorites_full <- meteorites_full[, c(2,4,7,8,6,10,9,5)]
colnames(meteorites_full) <- c('Name',
                               'Valid',
                               'Fell_or_Found',
                               'Year',
                               'Mass',
                               'longitude',
                               'latitude',
                               'recclass')


#UI
ui <- fluidPage(
    titlePanel("Meteorite Data Analysis"),
    fluidRow(
        tabsetPanel(
            tabPanel("About",
                     sidebarLayout(
                         sidebarPanel(
                             h1("Meteorite Data Analysis"),
                             h2("Visualizations by Matthew Younce"),
                             h3("PSDS Capstone")),
                         mainPanel(
                             p(),
                             p("This analysis has been performed on a dataset of meteorites from the Meteoritical Society via NASA."),
                             div("Here is a brief description of what you will find in each tab."),
                             hr(),
                             h4("Tab 1: About"),
                             div("This page."),
                             h4("Tab 2: Meteorite Data"),
                             div("This tab shows a map and table of the meteorite data after data scrubbing was complete."),
                             h4("Tab 3: KModes Clustering"),
                             div("This is the results of an unsupervised clustering.  KModes clustering is similar to KMeans clustering, except it is for categorical data."),
                             h4("Tab 4: Land Cover PDFs"),
                             div("This shows how likely a particular mass of meteorite is for a given land cover type. If the peak is more to the left, its more likely a meteorite in that land cover type will be less massive."),
                             h4("Tab 5: Falls over Time"),
                             div("This is an attempt to see if there is any change in the rate of meteorites observed falling.  While the number has increased recently, this may be due to better record keeping rather than anything else."),
                             h4("Tab 6: Falls vs. Finds"),
                             div("This shows whether a 1°x1° grid square has seen a fall event, a find, or both."),
                             h4("Tab 7: Correlation Tester"),
                             div("This allows a comparison of numeric variables in the dataset."),
                             h4("Tab 8: Full Dataset"),
                             div("This is similar to Tab 2, except this is the full dataset from NASA without any data scrubbing.")))),
            tabPanel("Meteorite Data",
                     sidebarLayout(
                         sidebarPanel(
                             sliderInput("yearslider",
                                          label = "Year",
                                          min = 800, 
                                          max = 2021,
                                          step = 5,
                                          value = c(800,2021),
                                          sep=""),
                             sliderInput("massslider",
                                         label = "Mass (Kg)",
                                         min = 0, 
                                         max = 65000,
                                         value = c(0,65000)),
                             checkboxGroupInput(inputId = "fellfoundcheckbox",
                                                label = "Fall or Find:",
                                                inline = TRUE,
                                                selected = c("Fell", "Found"),
                                                choiceNames = c("Observed Falling","Found later"),
                                                choiceValues = c("Fell", "Found")),
                             checkboxGroupInput(inputId = "typecheckbox",
                                                label = "Type of Meteorite:",
                                                inline = TRUE,
                                                selected = c("Stony","Stony-Iron","Iron","-"),
                                                choiceNames = c("Stony","Stony-Iron","Iron","Unknown"),
                                                choiceValues= c("Stony","Stony-Iron","Iron","-")),
                             checkboxGroupInput(inputId = "chondritecheckbox",
                                                label = "Chondrite or Achondrite:",
                                                inline = TRUE,
                                                selected = c("Chondrite","Achondrite","-"),
                                                choiceNames = c("Chondrite","Achondrite","Unknown"),
                                                choiceValues = c("Chondrite","Achondrite","-")),
                             checkboxGroupInput(inputId = "meteoriteLCcheckbox",
                                                label = "Land Cover at impact site:",
                                                selected = LC_names2,
                                                choiceNames = LC_names2,
                                                choiceValues = LC_names2)),
                         mainPanel(
                             leafletOutput("meteoritemap"),
                             dataTableOutput('meteorite_table')))),
            tabPanel("KModes Clustering",
                     sidebarLayout(
                         sidebarPanel(
                             radioButtons(
                                 inputId="KMradio",
                                 label="Choose cluster to display",
                                 selected = NULL,
                                 inline = TRUE,
                                 choices = c(0:6))),
                         mainPanel(
                             leafletOutput("kmodesmap"),
                             dataTableOutput("kmodestable")))),
            tabPanel("Land Cover PDFs",
                     sidebarLayout(
                         sidebarPanel(checkboxGroupInput(inputId = "pdfcheckbox",
                                                         label = "Choose Land Covers to display",
                                                         selected = LC_values,
                                                         choiceNames = LC_names,
                                                         choiceValues = LC_values)),
                         mainPanel(plotOutput("pdfplot")))),
            tabPanel("Falls over Time",
                     sidebarLayout(
                         sidebarPanel(
                             checkboxInput(inputId = "norm_check",
                                           label = "Normalize?", 
                                           value = FALSE)),
                         mainPanel(dygraphOutput("timelineplot"),
                                   plotOutput("timelinecorrplot")))),
            tabPanel("Falls vs. Finds",
                     sidebarLayout(
                         sidebarPanel(
                             checkboxGroupInput(inputId = "gridsquares",
                                                label = "Display 1° x 1° squares with:",
                                                inline = TRUE,
                                                selected = c("fall", "find", "both"),
                                                choiceNames = c("Falls Only (Red)", "Finds Only(Blue)", "Both (Green)"),
                                                choiceValues = c("fall", "find", "both"))),
                         mainPanel(leafletOutput("gsmap"),
                                   div("Finds Only: 1063, Falls Only: 712, Both: 181")))),
            tabPanel("Correlation Tester",
                     fluidRow(
                         column(6,wellPanel(
                             selectInput("corrvariable1",
                                         "Select variable #1:",
                                         c(list("Variables" = corrlist)),
                                         selected = "mass"))),
                         column(6,wellPanel(
                             selectInput("corrvariable2",
                                         "Select variable #2:",
                                         c(list("Variables" = corrlist)),
                                         selected = "lc_sample"))),
                         plotOutput("plot_chooseyourown"))),
            tabPanel("Full Dataset",
                     sidebarLayout(
                         sidebarPanel(
                             sliderInput("bonusyearslider",
                                         label = "Year",
                                         min = 800, 
                                         max = 2021,
                                         step = 5,
                                         value = c(800,2021),
                                         sep=""),
                             sliderInput("bonusmassslider",
                                         label = "Mass (Kg)",
                                         min = 0, 
                                         max = 65000,
                                         value = c(0,65000)),
                             checkboxGroupInput(inputId = "bonusvalid",
                                                label = "Valid or Relict:",
                                                inline = TRUE,
                                                selected = c("Valid", "Relict"),
                                                choiceNames = c("Valid","Relict"),
                                                choiceValues = c("Valid", "Relict")),
                             checkboxGroupInput(inputId = "bonusfellfoundcheckbox",
                                                label = "Fall or Find:",
                                                inline = TRUE,
                                                selected = c("Fell", "Found"),
                                                choiceNames = c("Observed Falling","Found later"),
                                                choiceValues = c("Fell", "Found"))),
                         mainPanel(
                             leafletOutput("bonusmeteoritemap"),
                             dataTableOutput('bonusmeteorite_table')))))))

##SERVER
server <- function(input, output, session) {
    
    reac <- reactiveValues(yearslider = c(800,2021),
                           fellfoundcheckbox = c("Fell", "Found"), 
                           typecheckbox = c("Stony","Stony-Iron","Iron","-"),
                           chondritecheckbox = c("Chondrite","Achondrite","-"),
                           massslider = c(0,65000000),
                           meteoriteLCcheckbox = LC_names2,
                           corrvariable1 = "mass",
                           corrvariable2 = "lc_sample",
                           bonusyearslider = c(800,2021),
                           bonusmassslider = c(0,65000),
                           bonusfellfoundcheckbox = c("Fell", "Found"),
                           bonusvalid = c("Valid","Relict"))
    
    observeEvent (input$yearslider, {reac$yearslider = input$yearslider})
    observeEvent (input$fellfoundcheckbox, {reac$fellfoundcheckbox = input$fellfoundcheckbox})
    observeEvent (input$typecheckbox, {reac$typecheckbox = input$typecheckbox})
    observeEvent (input$chondritecheckbox, {reac$chondritecheckbox = input$chondritecheckbox})
    observeEvent (input$massslider, {reac$massslider = input$massslider *1000})
    observeEvent (input$meteoriteLCcheckbox, {reac$meteoriteLCcheckbox = input$meteoriteLCcheckbox})
    observeEvent (input$corrvariable1, {reac$corrvariable1 = input$corrvariable1})
    observeEvent (input$corrvariable2, {reac$corrvariable2 = input$corrvariable2})
    observeEvent (input$bonusyearslider, {reac$bonusyearslider = input$bonusyearslider})
    observeEvent (input$bonusmassslider, {reac$bonusmassslider = input$bonusmassslider *1000})
    observeEvent (input$bonusfellfoundcheckbox, {reac$bonusfellfoundcheckbox = input$bonusfellfoundcheckbox})
    observeEvent (input$bonusvalid, {reac$bonusvalid = input$bonusvalid})
    
    output$pdfplot = renderPlot({
        pdf_plot <- subset(pdfmelt, variable %in% input$pdfcheckbox)
        p <- ggplot()
        p <- p + geom_line(data = pdf_plot, aes(x = x_axis, y = value, color=variable, group=variable))
        p <- p + ggtitle("Probability Density Function of Mass by Land Cover Type")
        p <- p + labs(x = "Meteorite Mass (g)", y= NULL)
        p})
    
    output$meteorite_table = renderDataTable({
        map_meteorites <- meteorites
        map_meteorites <- subset(map_meteorites, Fell_or_Found %in% reac$fellfoundcheckbox)
        map_meteorites <- map_meteorites[map_meteorites$Year >= reac$yearslider[1] & map_meteorites$Year <= reac$yearslider[2],]
        map_meteorites <- map_meteorites[map_meteorites$Mass >= reac$massslider[1] & map_meteorites$Mass <= reac$massslider[2],]
        map_meteorites <- subset(map_meteorites, Type %in% reac$typecheckbox)
        map_meteorites <- subset(map_meteorites, Chondrite %in% reac$chondritecheckbox)
        map_meteorites <- subset(map_meteorites, Land_Cover %in% reac$meteoriteLCcheckbox)
        map_meteorites})

    output$meteoritemap = renderLeaflet({
        map_meteorites <- meteorites
        map_meteorites <- subset(map_meteorites, Fell_or_Found %in% reac$fellfoundcheckbox)
        map_meteorites <- map_meteorites[map_meteorites$Year >= reac$yearslider[1] & map_meteorites$Year <= reac$yearslider[2],]
        map_meteorites <- map_meteorites[map_meteorites$Mass >= reac$massslider[1] & map_meteorites$Mass <= reac$massslider[2],]
        map_meteorites <- subset(map_meteorites, Type %in% reac$typecheckbox)
        map_meteorites <- subset(map_meteorites, Chondrite %in% reac$chondritecheckbox)
        map_meteorites <- subset(map_meteorites, Land_Cover %in% reac$meteoriteLCcheckbox)
        map <- leaflet(map_meteorites) %>% addTiles() %>% addMarkers(label = ~Name, clusterOptions = markerClusterOptions())
        map})
    
    output$bonusmeteorite_table = renderDataTable({
        map_meteorites <- meteorites_full
        map_meteorites <- map_meteorites[map_meteorites$Year >= reac$bonusyearslider[1] & map_meteorites$Year <= reac$bonusyearslider[2],]
        map_meteorites <- map_meteorites[map_meteorites$Mass >= reac$bonusmassslider[1] & map_meteorites$Mass <= reac$bonusmassslider[2],]
        map_meteorites <- subset(map_meteorites, Fell_or_Found %in% reac$bonusfellfoundcheckbox)
        map_meteorites <- subset(map_meteorites, Valid %in% reac$bonusvalid)
        map_meteorites})

    output$bonusmeteoritemap = renderLeaflet({
        map_meteorites <- meteorites_full
        map_meteorites <- map_meteorites[map_meteorites$Year >= reac$bonusyearslider[1] & map_meteorites$Year <= reac$bonusyearslider[2],]
        map_meteorites <- map_meteorites[map_meteorites$Mass >= reac$bonusmassslider[1] & map_meteorites$Mass <= reac$bonusmassslider[2],]
        map_meteorites <- subset(map_meteorites, Fell_or_Found %in% reac$bonusfellfoundcheckbox)
        map_meteorites <- subset(map_meteorites, Valid %in% reac$bonusvalid)
        map <- leaflet(map_meteorites) %>% addTiles() %>% addMarkers(label = ~Name, clusterOptions = markerClusterOptions())
        map})
    
    output$timelineplot <- renderDygraph({
        if (input$norm_check == FALSE) {
            falls_decade_plot <- subset(falls_decade, select = -c(falls_per_1M))
            dygraph(falls_decade_plot, main="Meteorite Falls", height=400) %>%
            dyOptions(drawGapEdgePoints = TRUE,fillGraph = TRUE,drawGrid=FALSE) %>%
            dyLegend(width = 400) %>%
            dySeries("meteorite_falls", label = "Falls") %>%
            dyAxis("y", label = "Meteorite Falls")  %>%
            dySeries("population", axis = "y2", label = "Population") %>%
            dyAxis("y2", label = "Population", independentTicks =TRUE)  %>%
            dyRangeSelector(height = 75)
        } else {
            falls_decade_plot <- subset(falls_decade, select = -c(meteorite_falls))
            dygraph(falls_decade_plot, main="Meteorite Falls (Normalized per 1 Million Population)", height=400) %>%
            dyOptions(drawGapEdgePoints = TRUE,fillGraph = TRUE,drawGrid=FALSE) %>%
            dyLegend(width = 400) %>%
            dySeries("falls_per_1M", label = "Normalized Falls") %>%
            dyAxis("y", label = "Meteorite Falls")  %>%
            dySeries("population", axis = "y2", label = "Population") %>%
            dyAxis("y2", label = "Population", independentTicks =TRUE)  %>%
            dyRangeSelector(height = 75)}})
    
    output$timelinecorrplot = renderPlot({
        if (input$norm_check == FALSE) {
            falls_decade_plot <- subset(falls_decade, select = -c(falls_per_1M))          
            corrplot <- ggplot(falls_decade_plot, aes(x=population, y=meteorite_falls)) + geom_point()
            corrplot <- corrplot + geom_smooth(method="auto", se=TRUE, fullrange=FALSE, level=0.95)
            corrplot
        } else {
            falls_decade_plot <- subset(falls_decade, select = -c(meteorite_falls))         
            corrplot <- ggplot(falls_decade_plot, aes(x=population, y=falls_per_1M)) + geom_point()
            corrplot <- corrplot + geom_smooth(method="auto", se=TRUE, fullrange=FALSE, level=0.95)
            corrplot}})
    
    output$kmodesmap = renderLeaflet({
        kmodes_filtered <- subset(kmodes, Cluster == input$KMradio)
        map_km <- leaflet(kmodes_filtered) %>% addTiles() %>% addMarkers(clusterOptions = markerClusterOptions())
        map_km})
    
    output$kmodestable =  renderDataTable({
        kmodes_filtered <- subset(kmodes, Cluster == input$KMradio)
        kmodes_filtered})
    
    output$gsmap = renderLeaflet({
        gsplot <- subset(gsfellfound, fallorfind %in% input$gridsquares)
        gsplot <- gsplot[-c(1) ] 
        colnames(gsplot) <- c("latitude", "longitude", "score", "group")
        getColor <- function(gsplot) {
            sapply(gsplot$score, function(score) {
                if(score == -1) {
                    "blue"
                } else if(score == 1) {
                    "red"
                } else {
                    "green"}})}
        icons <- awesomeIcons(
            icon = 'ios-close',
            iconColor = 'black',
            library = 'ion',
            markerColor = getColor(gsplot))
        map <- leaflet(gsplot) %>% addTiles() %>% addAwesomeMarkers(~longitude, ~latitude, icon=icons)
        map})
    
    output$plot_chooseyourown = renderPlot({
        corr_final <- meteorites_corr %>% select(reac$corrvariable1, reac$corrvariable2)
        p <- ggplot(corr_final, aes(x=corr_final[[1]], y=corr_final[[2]]))
        p <- p + geom_point(color="firebrick")
        p <- p + geom_smooth(method = lm, se = FALSE, color="blue") 
        p <- p + ggtitle(paste0(reac$corrvariable1," vs. ", reac$corrvariable2,"\nCorrelation Coeff. = ",cor(corr_final)[1,2]))
        p <- p + labs(x = reac$corrvariable1, y= reac$corrvariable2)
        p})
}

## Run App
shinyApp(ui = ui, server = server)

In [49]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "Capstone" #This is to specify the course path for the shiny server
folder <- "meteoriteanalysis" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

[1] "sending incremental file list"                                                                                       
[2] "meteoriteanalysis/"                                                                                                  
[3] "meteoriteanalysis/app.R"                                                                                             
[4] "\r         23,897 100%    0.00kB/s    0:00:00  \r         23,897 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=132/135)"
[5] ""                                                                                                                    
[6] "sent 28,049 bytes  received 55 bytes  56,208.00 bytes/sec"                                                           
[7] "total size is 88,101,765  speedup is 3,134.85"

In [42]:
url <- cat("https://shiny.sgn.missouri.edu/students/ymw.0780/",course,"/",folder,"\n\n",sep="")
print(url)

https://shiny.sgn.missouri.edu/students/ymw.0780/Capstone/meteoriteanalysis

NULL


In [ ]:
meteorites[meteorites$Mass == 60000000,]

In [ ]:
meteorites <- read.csv(file='data/results/meteorites_lc.csv')
meteorites <- meteorites[, c(2,3,7,8,6,10,9,18,13:16,12,5)]
colnames(meteorites) <- c('id','Meteorite Name','Fell_or_Found','Year', 'Mass (g)', 'Longitude', 'Latitude', 'Land Cover', 'Chondrite', 'Type', 'Level 3', 'Level 4', 'Group','recclass')

meteorites

In [ ]:
pdf <- read.csv(file = 'data/results/pdf.csv')
pdf

In [ ]:
library(reshape2)
pdfmelt<-melt(pdf,id="x_axis")
print(pdfmelt)

In [ ]:
library(ggplot2)

pdfcheckbox <- list("tropical_forest_pdf","water_pdf","tundra_pdf")

pdf_plot <- subset(pdfmelt, variable %in% pdfcheckbox)

p <- ggplot()
p <- p + geom_line(data = pdf_plot, aes(x = x_axis, y = value, color=variable, group=variable))
p <- p + ggtitle(paste0("Probability Density Function of Mass by Land Cover Type"))
p <- p + labs(x = "Meteorite Mass (g)", y= NULL)
p

pdf_plot

In [ ]:
fellfoundcheckbox = c('Fell')
yearslider = c(500,1500)





map_meteorites <- meteorites
map_meteorites <- subset(map_meteorites, Fell_or_Found %in% fellfoundcheckbox)
map_meteorites <- map_meteorites[map_meteorites$Year >= yearslider[1] & map_meteorites$Year <= yearslider[2],]

map_meteorites

In [ ]:
falls_decade

In [ ]:
meteorites_dummy <- read.csv(file='data/results/meteorites_dummy.csv')
corr_final <- meteorites_dummy %>% select("mass", "lc_sample")
head(corr_final)

In [ ]:
p <- ggplot(corr_final, aes(x=corr_final[[1]], y=corr_final[[2]]))
p <- p + geom_point(color="firebrick")
p <- p + geom_smooth(method = lm, se = FALSE, color="blue") 
p <- p + ggtitle(paste0("reac$choose1"," vs. ", "reac$choose2","\nCorrelation Coeff. = ",cor(corr_final)[1,2]))
p <- p + labs(x = "reac$choose1", y= "reac$choose2")
p
    

In [ ]:
colnames(meteorites)

In [ ]:
unique(meteorites$Group)

In [ ]:
kmodes <- read.csv(file='data/results/meteorites_clustered.csv')


In [ ]:
kmodes

In [ ]:
summary(kmodes)

In [ ]:
km0 <- subset(kmodes, kmodes$Cluster == 0)
km0 <- subset(km0,select=-c(1))
km1 <- subset(kmodes, kmodes$Cluster == 1)
km1 <- subset(km1,select=-c(1))
km2 <- subset(kmodes, kmodes$Cluster == 2)
km2 <- subset(km2,select=-c(1))
km3 <- subset(kmodes, kmodes$Cluster == 3)
km3 <- subset(km3,select=-c(1))
km4 <- subset(kmodes, kmodes$Cluster == 4)
km4 <- subset(km4,select=-c(1))
km5 <- subset(kmodes, kmodes$Cluster == 5)
km5 <- subset(km5,select=-c(1))
km6 <- subset(kmodes, kmodes$Cluster == 6)
km6 <- subset(km6,select=-c(1))


In [ ]:
summary(km0)

In [ ]:
library(tidyverse)

kmodes2<- kmodes

kmodes2 <- kmodes %>% 
  group_by(Cluster) %>% 
  summarize(min_mass = min(mass),
            max_mass = max(mass),
            min_year=min(year),
            max_year=max(year),
            min_long=min(longitude),
            max_long=max(longitude),
            min_lat=min(latitude),
            max_lat=max(latitude))
kmodes2
            
kmodes2$stony <- count(type)


kmodes3<- kmodes %>%
  group_by(Cluster) %>% 
  count(type)

kmodes3

kmodes4 <- kmodes %>%
  group_by(Cluster) %>% 
  count(chondrite)

kmodes4

total <- merge(kmodes4,kmodes3,by="Cluster")

total

In [ ]:
kmodesA <- kmodes %>% count(Cluster,type,chondrite)

In [ ]:
kmodesz <- as.table(kmodes)

In [ ]:
dim(meteorites)

In [ ]:
            map_meteorites <- subset(meteorites, Fell_or_Found %in% c("Fell","Found"))
#            map_meteorites <- map_meteorites[map_meteorites$Year >= reac$yearslider[1] & map_meteorites$Year <= reac$yearslider[2],]
#            map_meteorites <- map_meteorites[map_meteorites$Mass >= reac$massslider[1] & map_meteorites$Mass <= reac$massslider[2],]
#            map_meteorites <- subset(map_meteorites, Type %in% reac$typecheckbox)
#            map_meteorites <- subset(map_meteorites, Chondrite %in% reac$chondritecheckbox)
#            map_meteorites <- subset(map_meteorites, Land_Cover %in% reac$meteoriteLCcheckbox)


In [ ]:
dim(map_meteorites)

In [ ]:
meteorites[meteorites$Land_Cover == "Pasture/Land used for Grazing",]

In [ ]:
meteorites <- read.csv(file='data/results/meteorites_lc.csv',stringsAsFactors = FALSE)
meteorites <- meteorites[, c(3,7,8,6,10,9,18,13:16,12,5)]
colnames(meteorites) <- c('Name','Fell_or_Found','Year', 'Mass', 'longitude', 'latitude', 'Land_Cover', 'Chondrite', 'Type', 'Level_3', 'Level_4', 'Group','recclass')


In [ ]:
unique(meteorites['Land_Cover'],)

In [ ]:
str(meteorites$Land_Cover)


In [ ]:
meteorites$Land_Cover<-replace(meteorites$Land_Cover, meteorites$Land_Cover=="Pasture/Land used for\nGrazing", "Pasture/Land used for Grazing")

In [ ]:
head(meteorites)

In [ ]:
write.csv(meteorites,file='dashboard/data/results/meteorites_lc.csv')